<a href="https://colab.research.google.com/github/pavlosemyvolos/Assessment/blob/main/Lab6_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!tar xf spark-3.0.0-bin-hadoop3.2.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.0.0-bin-hadoop3.2'

!pip install -q findspark

In [12]:
!pip3 install pyspark==3.0.0

In [13]:
import findspark
findspark.init("/content/spark-3.0.0-bin-hadoop3.2")
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col, count, avg
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

Exception: ignored

In [ ]:
spark = (SparkSession.builder.master("local").appName("WordCount").config("spark.some.config.option", "some-value").getOrCreate())

In [ ]:
!spark-submit --version

In [ ]:
readAkas = spark.read.csv("akas.tsv", sep=r'\t',header=True)
readAkas.show()

In [ ]:
readName_Basics = spark.read.csv("name_basics.tsv", sep=r'\t',header=True)
readName_Basics.show()

In [ ]:
readTitle_Basics = spark.read.csv("title_basics.tsv", sep=r'\t',header=True)
readTitle_Basics.show()

In [ ]:
readEpisode = spark.read.csv("episode.tsv", sep=r'\t',header=True)
readEpisode.show()

In [ ]:
readCrew = spark.read.csv("crew.tsv", sep=r'\t',header=True)
readCrew.show()

In [ ]:
Ratings = spark.read.csv("ratings.tsv", sep=r'\t',header=True)
Ratings.show()

In [ ]:
spark = SparkSession.builder.appName("example").getOrCreate()
df = pd.DataFrame({'First_Name': ['Pavlo', 'Sergiy', 'Mtkola'], 'Last_Name': ['Semyvolos','Konstantionov','Zdoryk']})
schema = StructType(fields=[
    StructField("First_Name", StringType()),
    StructField("Last_Name", StringType())
])
spark_df = spark.createDataFrame(df, schema=schema)
spark_df.show()

In [ ]:
spark = SparkSession.builder.appName("example").getOrCreate()

schema = StructType(fields=[
    StructField("nconst", StringType(), nullable=True),
    StructField("primaryName", StringType(), nullable=True),
    StructField("birthYear", DoubleType(), nullable=True),
    StructField("deathYear", StringType(), nullable=True),
    StructField("primaryProfession", StringType(), nullable=True),
    StructField("knownForTitles", StringType(), nullable=True)
])

name_basics_df = spark.read.csv("basics.tsv", sep='\t', nullValue='\\N', header=True, schema=schema)

name_basics_df.printSchema()
name_basics_df.show(7, truncate=False)

In [ ]:
Akas = spark.read.csv("akas.tsv", sep=r'\t',header=True)
Akas.show()

In [ ]:
ukrainian_titles = Akas.filter((col("language") == "uk") | (col("language") == "UA")).select("title")
ukrainian_titles.show()

In [ ]:
output_path = "output1.csv"
ukrainian_titles.write.mode("overwrite").option("header", "true").csv(output_path)

In [ ]:
First = spark.read.csv("output1.csv", sep=r',',header=True)
First.show()

In [ ]:
readTitle_Basics.show(7, truncate=False)
Ratings.show(7, truncate=False)

In [ ]:
merged_data = readTitle_Basics.join(Ratings, 'tconst')

windowSpec = Window.partitionBy('genres').orderBy(F.desc('numVotes'))

ranked_movies = merged_data.withColumn('rank', F.dense_rank().over(windowSpec))

top_movies_by_genre = ranked_movies.filter(F.col('rank') <= 10) \
    .select('genres', 'primaryTitle', 'averageRating', 'numVotes')

top_movies_by_genre.show(truncate=False)